In [ ]:
%load_ext autoreload
%autoreload 2

# sweep.ipynb

import numpy as np
from datetime import datetime

from SQDMetal.COMSOL.Model import COMSOL_Model
from adjoint_sim_sf.ParametricDesign import SymmetricTransmonDesign
from adjoint_sim_sf.AdjointSolver import AdjointEvaluator
from adjoint_sim_sf.Optimiser import Optimiser
from adjoint_sim_sf import Experiment


# 1) COMSOL engine
if COMSOL_Model._engine is None:
    COMSOL_Model.init_engine()

# 2) Evaluator config (must match AdjointEvaluator.update_params / attributes)
base_config = {
    "freq_value": 8e9,
    "num_adj_sample_points": 20,
    "param_perturbation": [1e-5],
    "fwd_source_strength": 1e-2,
    "adjoint_rotation": float(np.pi / 2),
}

# 3) Objects
designer  = SymmetricTransmonDesign()
evaluator = AdjointEvaluator(designer, config=base_config)

# (Optional) deterministic sampling for EPR sources across sweeps
evaluator.random_seed = 12345

# Default folder: exp_YYYYMMDD-HH%M-%S (comes from Experiment.__init__)
exp = Experiment()


x_vals = np.linspace(0.18, 0.23, 80, float)
y_vals = np.linspace(0.25, 0.25, 1, float)
param_grid = np.array([[x, y] for x in x_vals for y in y_vals], dtype=float)
initial_params = np.array([0.20, 0.20], float)

# 5) Optimiser
initial_params = np.array([0.20, 0.20], float)
lr = 0.01
opt = Optimiser(initial_params=initial_params, lr=lr, evaluator=evaluator)

# 6) Sweep different JJ/EPR weightings
w_jj = 0.5
timestamp = datetime.now().strftime("%Y%m%d-%H%M-%S")

# Save the run-level config once
exp.save_config({
    "adjoint_evaluator": evaluator.to_config_dict(),
    "grid": {"x_vals": x_vals.tolist(), "y_vals": y_vals.tolist()},
    "initial_params": initial_params.tolist(),
    "lr": lr,
    "notebook_timestamp": timestamp,
})

filename="results.jsonl"

results = exp.stream_results(
    opt.sweep_multi_objective(param_grid, w_jj=0.5, verbose=True),
    filename="results.jsonl",
    verbose=True
)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Evaluating point 0/80: [0.18 0.25], w_jj=0.5
jj: 1 sources, loss=-4.643e+14
sa: 20 sources, loss=3.332e+22
[0] Saved result to results.jsonl
    loss=1.665966e+22
Evaluating point 1/80: [0.18063291 0.25      ], w_jj=0.5
jj: 1 sources, loss=-5.267e+14
sa: 20 sources, loss=3.502e+22
[1] Saved result to results.jsonl
    loss=1.751222e+22
Evaluating point 2/80: [0.18126582 0.25      ], w_jj=0.5
jj: 1 sources, loss=-6.400e+14
sa: 20 sources, loss=4.918e+22
[2] Saved result to results.jsonl
    loss=2.459219e+22
Evaluating point 3/80: [0.18189873 0.25      ], w_jj=0.5
jj: 1 sources, loss=-7.078e+14
sa: 20 sources, loss=3.369e+22
[3] Saved result to results.jsonl
    loss=1.684449e+22
Evaluating point 4/80: [0.18253165 0.25      ], w_jj=0.5
jj: 1 sources, loss=-8.084e+14
sa: 20 sources, loss=3.490e+22
[4] Saved result to results.jsonl
    loss=1.744903e+22
Evaluating point 5/80: [0.18316456 0.25      ], w

In [32]:

import matplotlib.pyplot as plt
import numpy as np

records = exp.load_jsonl(filename)
print(len(records))
print(records[0])


x = [r['params'][0] for r in records]
grad = [r["grad_jj"][0] *2e-15 for r in records]

# centre loss around zero
mean_loss = np.mean([r["loss_jj"] for r in records])
loss = [r["loss_jj"] - mean_loss for r in records]

def numerical_gradient(x, y):
    grad = []
    for i in range(1, len(y)-1):
        dy = y[i+1] - y[i-1]
        dx = x[i+1] - x[i-1]
        grad.append(dy/dx)
    grad = [grad[0]] + grad + [grad[-1]]
    return grad

num_grad = numerical_gradient(x, loss)


fig, ax1 = plt.subplots()
ax1.plot(x, grad, color='b', label='Gradient (JJ)')
ax1.plot(x, loss, color='r', label='Loss')

ax2 = ax1.twinx()
ax2.plot(x, num_grad, color='g', label='Numerical Gradient (Loss)', marker='x')

plt.show()

20
{'params': [0.18, 0.25], 'loss': 1.6659663620858005e+22, 'grad': [4.838603041874597e+30, -1.371706187246267e+30], 'loss_jj': -464316903604136.94, 'loss_sa': 3.3319327706032913e+22, 'loss_ma': 0.0, 'grad_jj': [5.682378480247212e+28, 5.165827350233496e+28], 'grad_sa': [9.620382298946722e+30, -2.7950706479948687e+30], 'grad_ma': [0.0, 0.0], 'index': 0, 'w_jj': 0.5, 'w_sa': 0.5}


In [31]:

epr_type = "sa"
# epr_type = "jj"


x = [r['params'][0] for r in records]

grad = [r[f"grad_{epr_type}"][0]for r in records]

loss = [r[f"loss_{epr_type}"]for r in records]

def numerical_gradient(x, y):
    grad = []
    for i in range(1, len(y)-1):
        dy = y[i+1] - y[i-1]
        dx = x[i+1] - x[i-1]
        grad.append(dy/dx)
    grad = [grad[0]] + grad + [grad[-1]]
    return grad

num_grad = numerical_gradient(x, loss)

def rescale(data):
    min_data = min(data)
    max_data = max(data)
    return [(d - min_data) / (max_data - min_data) for d in data]

grad_rescale = rescale(grad)
loss_rescale = rescale(loss)
num_grad_rescale = rescale(num_grad)



fig, ax1 = plt.subplots()
ax1.plot(x, grad_rescale, color='b', label=f'Gradient ({epr_type})')
ax1.plot(x, loss_rescale, color='r', label='Loss')

ax2 = ax1.twinx()
ax2.plot(x, num_grad_rescale, color='g', label='Numerical Gradient (Loss)', marker='x')

plt.show()

for r in records:
    print(r["grad_sa"])

print(grad_std)

[9.620382298946722e+30, -2.7950706479948687e+30]
[2.1087025396705017e+31, -9.279240638677037e+29]
[1.758904749724474e+31, 6.333504002992282e+30]
[-1.0248569227865363e+32, 5.077675868511335e+29]
[-1.5835708923067545e+32, 4.3944763880893285e+30]
[-3.1036914620971065e+31, 5.009044680422408e+29]
[-1.5434036069018318e+32, -5.166328908057224e+31]
[-1.3915948753881694e+33, 6.512473676182435e+31]
[-1.1295561717405859e+33, -2.9564782172973138e+31]
[-1.332830264925174e+33, -1.5130979326793121e+32]
[9.145271048085184e+33, -2.390563966566033e+30]
[-1.7528481275286285e+31, -2.7917109268687064e+32]
[4.112938639018317e+33, -6.580705304054673e+31]
[-2.6002294623611927e+31, -4.721968041244138e+31]
[-1.1527230055739838e+31, -2.499953932133451e+31]
[-9.853483135636036e+30, -2.2394653768421754e+31]
[-4.297929055826788e+30, -1.2763877867948468e+31]
[-2.2149487216620533e+30, -6.979741331123302e+30]
[-1.226069638366257e+30, 7.906727483186915e+29]
[-8.234738731083445e+29, 2.762333872880766e+29]
2.254038524435

In [8]:
from adjoint_sim_sf import Experiment, Plotter
import numpy as np

import matplotlib.pyplot as plt

# Load the file that was just created in the first cell
rel_filename = results_filename
data = exp.load_jsonl(rel_filename)
print(len(data))
print(data[0].keys())

# Convert to arrays
params_array = np.array([r["params"] for r in data])
loss_array = np.array([r["loss"] for r in data])
adj_grad_array = np.array([r["grad"] for r in data])

# Compute numeric gradients
grads = exp.compute_numeric_grads(
    params_array=params_array,
    loss_array=loss_array,
    adj_grad_array=adj_grad_array,
)

# Contour plots
fig1, axes1 = Plotter.gradient_contour_2d(
    all_params=params_array,
    loss_array=loss_array,
    numeric_positions=grads["valid_params"],
    numeric_gradients=grads["numeric_grad_array"],
    adj_positions=grads["valid_params"],
    adj_gradients=grads["valid_adj_grad_array"],
    title="Loss Contours: Numeric vs Adjoint",
)
plt.show()

# Magnitude comparison
fig2, ax2 = Plotter.gradient_mag_comparison(
    numeric_gradients=grads["numeric_grad_array"],
    adjoint_gradients=grads["valid_adj_grad_array"],
    scale=0.02,
    uselog=True,
    title="Gradient Magnitude Comparison",
)
plt.show()

4
dict_keys(['params', 'loss', 'grad', 'loss_jj', 'loss_sa', 'loss_ma', 'grad_jj', 'grad_sa', 'grad_ma'])
Computed 1 numerical gradients
